# Mighty Analyst Test 

This document aims to provide answers to the Mighty Analyst Case provided by the team of MyTomorrows. It consists of 3 parts: 
- **Task 1:** Providing simple procedure statistics to a client
- **Task 2:** Providing information to specific user groups
- **Task 3:** Sharing information to senior management & other employees

The Case description mentions 3 stages of maturity when it comes to the data architecture: Excel files (included in the case), a postgres database connected to Tableau, and finally dbt. 

In this solution we will exclude the following: 
- Setting up a dbt model, as it does require some configuration time.

Instead we will use the following setup: 
- We will mimic the current architecture by uploading all files to a Google BigQuery postgres database, to simulate the current ecosystem
- This database will be used both in this notebook to create helper table, and in Tableau to create visualizations based on those helper tables


### Note
This data was assumed to be free of legal, privacy, security, and intellectual property restrictions, as mentioned in the [source](https://doi.org/10.1093/jamia/ocx079) present in  `input/citation.text`. As such, no precautions were taken to protext access to the data. In a real case scenario, more care would be taken to handle satisfactory security measures to protect patients and clients data.

### Initial Setup

In [1]:
# Installing required packages
%pip install --upgrade google-api-python-client --upgrade google-cloud --upgrade google-cloud-bigquery-storage google-auth db-dtypes pandas pandas-gbq pip install

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Setting up the Postgres database client for future queries
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
key_path = 'secrets/service_account_key.json'

# Create credentials using the service account key
credentials = service_account.Credentials.from_service_account_file(key_path)

# Create a BigQuery client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Execute a test query to showcase how it is used.
query = """
    SELECT * 
    FROM `mytomorrows-analyst-test.mytomorrows_edwingelebart.dm_organization`
"""
query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()

print(df)

                                     Id                            NAME  \
0  d78e84ec-30aa-3bba-a33a-f29a3a454662  MASSACHUSETTS GENERAL HOSPITAL   

           ADDRESS    CITY STATE   ZIP        LAT        LON  
0  55 FRUIT STREET  BOSTON    MA  2114  42.362813 -71.069187  


### Task 1: Providing simple procedure statistics to a client

**Description:** The client (United Healthcare) asks you to prep the data, and create some visualizations to get some more information on the data that is present. They would like to see the procedures ordered by total cost and and a visualization showing the amount of times procedures have been performed ordered from most held, to least held. We would also like you to give extracts of the data that is used in the visualtions (so the raw data behind the visualizations). Finally we would like to receive a graph showing patients, who follow the same procedures that are our top held procedures from our clients' patients. 

#### Solution

To enable these mulitiple graphs to be created, we will create a helper table providing us with all the information needed. 

In [3]:
# Create a helper table augmenting the procedure table, adding useful columns
with open('queries/helper_procedures.sql', 'r') as file:
    query = file.read()

query_job = client.query(query)

This table created a new table with the necessary columns. You can find the visualisations created for Task #1 in tab **Overview of Procedures** [here](https://public.tableau.com/app/profile/edwin.gelebart/viz/myTomororrows-MightyAnalystTest/OverviewofProcedure?publish=yes). It contains: 
1. A view showing both the total cost per procedure and the amount of times it was made.
2. A parameter allows the user to select which field is used to order the procedures.
3. In tab **Procedures - Raw Extract**, you can access and download the raw data used in the vizualisations. This can be done by using the _Download_ button on the top right of the screen.
4. A view showing other patients than the current client (United Healthcare) and the potential of their top procedures.

##### Note
- The base cost is assumed to be in dollars, without indications of unit. 
- Some procedure were found to have very similar names, with a few differences in the naming. Those could be combined, potentially showing a different ordering. In the absence of such request, I assumed those similar procedures should be considered as different
- The current dashboard technically includes the data of other clients. This is to show the way the dashboard works for all clients. To satisfy the asked feature, we could simply hide this filter from the dahsboard. More info on this client data access logic in Task #2.
- The last part of the assignment `we would like to receive a graph showing patients, who follow the same procedures that are our top held procedures from our clients' patients` is ambiguous. It was intepreted as showing the number of clients taking part on the top X procedures of the current client with competitors. 

### Task 2: Providing information to specific user groups

**Description:** Since we are already delivering a report to one of our clients, we cannot leave out the other client. They want to have information of the hospital doing the insurances and the amount of patients they help in a specific time frame. They consist of a quite a large team where each of the members has different specialty (procedure that they know everything of). They would like to easily select their own specialty as to not have to look at data of others. Its important to know that one of the specialties (hospice treatments) cannot be displayed to members with a different specialty.

#### Solution

There is quite some ambiguity in the question. We will decouple this question into multiple parts so that we can answer each element seperately: 

##### Part #1: Extending the report to other client
Such dashboards can be very useful for clients to understand their data, and guide them more towards understandibiltiy of patients needs. It is therefore crucial that all clients are enabled to do so. 
In this particular case, I would rely on 2 options: 
- **Set up access-controlled folders in the Tableau server** This setup would rely on central datasources accesible by all, 
which are filtered at the data ingestion stage of the tableau dashboard creation. This would lower the maintenance efforts on the data structure side, as all changes would be direclty effective to all clients.
- If more personalisation is needed for specific clients, we could **Set up parameterized data models in dbt or SQL**. This option would deduplicate the logic for backend tables per client, enabling custom logics and clearer boundaries of data storage. I would however advise using this setup in niche needs of clients, as the maintenance costs on both data modeling (dbt) and data visualization (tableau) would be high.

##### Part #2: Information on Hospital specific data
The current dataset contains only one organization or hospital: the Massachusetts General Hospital. It is included as being part of the filters on which one can filter on in the dashboard provided at Task #1. We could envision having more specific information available on hospital level. One option we can already do is to enable the dashboard user to select the level of detail they are interested in. That is why a parameter `Level of Detail` was introduced in the dashboard shared in Task #1, enabling users to select on which granularity the different metrics are needed. For now I have included Procedure (default), hopsitals, and patients. 

##### Part #3: Information on Patients Helped
This information was also added to the dashboard in Task #1, showing per procedure, how many distinct patients were helped, as well as an overall number on the right side. Date filters are also included, enabling users to select a specific time frame based on the start and end dates of procedures.

##### Part #4: Speciality-specific information
On top of adding a procedure filter, we could also control what data each team member sees, based on their role. Tableau has functionalities to implement specific data access on data rows (called Row Level Security), which enables us to rely on one interface for all users, making it easier to maintain, while ensuring that noone sees data that they shouldn't see. Here, implementing a list of allowed users enabled to see Hospice treatments, and integrating it into the dashboard could ensure that only this group sees this very critical information.

### Task 3: Sharing information to senior management & other employees
We would like to have reports for senior management, preferably with as much detail as possible. We would like to be able to have extracts of the visualized data as well. Only senior management is allowed to see these visualisations, whereas other employees in the org are allowed to have access to the raw extracts. One visualization should be about the data of our clients, how many patients we have helped from specific areas. Possibly to identify any relationship between the area of residence and the preocedures that are being performed. The other graphs we would like to see are non-client related, we would like to see if there are any patients grouped together that are not affeliated to any of our clients, but are in close proximity of patients who are in contact with our clients.

#### Solution

To enable the dual usage of this need, we will need to create a helper datatable in our postgres database, which is used by both usergroups in different dashboards. 

Tableau dashboards access can be managed by creating Access Controlled folders, enabling us to provide tailored view for different people within our organization. In this particular case, I would recommend doing 1 unique dashboard containing the views for all stakeholders, and hide specific views upon uploading to the different tableau folders. For the sake of this exercise, we will only create one dashboard. 

To help us in displaying these information, we will create a helper table for encounters, before linking it together with the patient table of our postgres database in Tableau. A patient can be affiliated to multiple clients of ours, so keeping that N-to-M relationship in Tableau can be useful. 

In [4]:
# Create a helper table augmenting the procedure table, adding useful columns
with open('queries/helper_encounters.sql', 'r') as file:
    query = file.read()

query_job = client.query(query)

The Tableau Dashboard can be found [here](). It contains: 
- A main view destined to the senior Management team, showing overall statistics on the clients ecosystem. It shows 
    - Simple metrics on number of patients, encounters, claim cost and client costs
    - A view showing per Zone the different metrics. It should help identify potential spatial opportunities. This view can be changed to show the right data by changing the metric used (one of the 4 presented above) and the granularity (Patient, Zip Code, cities).
    - A view showing the potential of clients in the Zip codes where they currently are operating. The size of the circle shows the Competitors market, while the color indicates the market share of our clients. The tool tip contains further information on the exact details.
- An extract view displaying all data used in the other views. Note: for other teams than the Senior Management team, this view will be the only one shown.

**Note:**
- Some lat/long points seem to be in the middle of the water on the map. A quick look on internet reveals that most of these addresses do not exist. It was therefore assumed this is due to low accuracy of this test data. These datapoints were not excluded, though it would be investigated further if it happened to be the case in production data. 
- The closeness of patients was determined whether they were part of the same zip code. We could with real data calculate the flying or driving distance and use those indicators as a better measure. 
